In [ ]:
import os

In [ ]:
#путь к папке с корпусом (лемматизированный корпус)
path = "corpus***"
dirs = os.listdir(path)

In [ ]:
i = 0
# словарь с именем файла и меткой класса (например, название рубрики или значение тональности)
dict_score = {}

In [ ]:
for file in dirs:
    #есть нетекстовые файлы
    if (file[-4:] == '.txt'):
        with open(path + '/' + file, 'r', encoding='utf-8') as readfile:
            for line in readfile:
                list_words = line.split()
                # последним токеном в файле идёт оценка = для текстов отзывов
                dict_score[file] = list_words[len(list_words)-1]
                # формируем файл для анализа
                with open('reviews_for_work.txt', 'a', encoding='utf-8') as wfile:
                    # первым токеном в файле идёт номер файла, он не нужен
                    wfile.write(' '.join(list_words[1:len(list_words) - 1]) + '\n')
                    i += 1

In [ ]:
print(f'files: {i}')

In [ ]:
print(f'dict_score: {len(dict_score)}')

In [ ]:
import re
import math
import pandas as pd
import nltk

In [ ]:
with open('Boyarsky.txt', 'r', encoding='utf-8') as file:
    f = file.read()

In [ ]:
text_words_2 = re.findall(r'[а-яёА-ЯЁ]+', f)

text_words_1 = []
for i in text_words_2:
    k = i.lower()
    text_words_1.append(k)

text_words_2

In [ ]:
text_words_1

In [ ]:
with open('swl.txt', 'r', encoding='utf8') as swl:
    swlist = []  # список стоп-слов из файла
    for i in swl:
        i = i.replace('\n','')
        swlist.append(i)
        # print(swlist)

len(swlist)

941

In [ ]:
text_words = []
for i in text_words_1:
    #i = morph.parse(i)[0].normal_form
    if i not in swlist:
        text_words.append(i)

text_words

In [ ]:
len(text_words)

2572

In [ ]:
frequency={}
for word in text_words:  # рассчитываем частоты слов
    count = frequency.get(word,0)  # создаём словарь СЛОВО и ЧАСТОТА
    frequency[word] = count + 1  # увеличиваем ЧАСТОТУ при считывании одинаковых лемм слова

frequency

In [ ]:
freq = sorted(frequency.items(), key=lambda x:x[1], reverse = True) #создаём список, в котором пары СЛОВО и ЧАСТОТА упорядочены по убыванию
length = len(text_words)

print(length)
print(freq)

2572
[('сущ', 84), ('прил', 59), ('предлог', 47), ('пад', 43), ('этим', 33), ('словосочетанием', 32), ('стоит', 32), ('род', 31), ('следует', 31), ('оборот', 31), ('словосочетания', 27), ('этого', 17), ('слов', 15), ('предложный', 15), ('словосочетаний', 10), ('слова', 10), ('глагол', 10), ('является', 9), ('словосочетание', 9), ('ними', 9), ('таким', 9), ('местоим', 9), ('рост', 9), ('союз', 9), ('деле', 8), ('оборотов', 8), ('рода', 8), ('был', 8), ('ним', 8), ('правее', 8), ('счет', 8), ('нкря', 7), ('связи', 7), ('согласованное', 7), ('концов', 6), ('время', 6), ('того', 6), ('оборотом', 6), ('качестве', 6), ('которые', 6), ('смерти', 6), ('рубеж', 6), ('классов', 6), ('класса', 6), ('имени', 6), ('всех', 6), ('точки', 6), ('лице', 6), ('свете', 6), ('предложений', 5), ('группы', 5), ('оборотами', 5), ('сказать', 5), ('обороты', 5), ('начала', 5), ('среднем', 5), ('свою', 5), ('справа', 5), ('поселения', 5), ('страны', 5), ('космос', 5), ('число', 5), ('области', 5), ('собака', 5),

In [ ]:
n_list=[]
for words in freq:  # проверяем каждую пару СЛОВО и ЧАСТОТА в отсортированном тексте
    fr_lst = list(words) # превращаем пары в круглых скобках, созданные анонимной функцией LAMBDA, в список из двух элементов - СЛОВО и ЧАСТОТА
    n_list.append(fr_lst)

n_list

In [ ]:
k = 1
for i in n_list:
    logfr = math.log(int(i[1]),10)
    logrank = math.log(k, 10)  # берём десятичные логарифмы для ранга и для частоты
    i.append(k)
    i.append(logfr)
    i.append(logrank)
    k+=1

In [ ]:
n_list

In [ ]:
frame = pd.DataFrame(n_list, columns=['Words','Frequencies','Ranks', 'Logarithm of Frequencies', 'Logarithm of ranks']) # собираем фрейм
frame.to_csv('Frequency Dictionary.csv',encoding='utf-8', sep=';', index=False) #экспортируем в файл

In [ ]:
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(text_words)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(text_words)

# bigrams
bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)
bigramFreqTable.to_csv('Bigrams.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл

# trigrams
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
trigramFreqTable.to_csv('Trigrams.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл

In [ ]:
bigram_freq

In [ ]:
# t-test
bigramTtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','T-test']).sort_values(by='T-test', ascending=False)
trigramTtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.student_t)), columns=['trigram','T-test']).sort_values(by='T-test', ascending=False)
bigramTtable = bigramTtable.head(20)
trigramTtable = trigramTtable.head(20) #вычленяем первые 20 лучших би- и триграмм
bigramTtable.to_csv('Bigrams-T-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл
trigramTtable.to_csv('Trigrams-T-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл

# chi-squared
bigramCHItable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','Chi-squared']).sort_values(by='Chi-squared', ascending=False)
trigramCHItable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','Chi-squared']).sort_values(by='Chi-squared', ascending=False)
bigramCHItable = bigramCHItable.head(20)
trigramCHItable = trigramCHItable.head(20) #вычленяем первые 20 лучших би- и триграмм
bigramCHItable.to_csv('Bigrams-CHI-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл
trigramCHItable.to_csv('Trigrams-CHI-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл

# log-likelihood
bigramlikelihoodtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.likelihood_ratio)), columns=['bigram','Log-Likelihood']).sort_values(by='Log-Likelihood', ascending=False)
trigramlikelihoodtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.likelihood_ratio)), columns=['trigram','Log-Likelihood']).sort_values(by='Log-Likelihood', ascending=False)
bigramlikelihoodtable = bigramlikelihoodtable.head(20)
trigramlikelihoodtable = trigramlikelihoodtable.head(20) #вычленяем первые 20 лучших би- и триграмм
bigramlikelihoodtable.to_csv('Bigrams-likelihood-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл
trigramlikelihoodtable.to_csv('Trigrams-likelihood-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл

# pmi
bigramPMItable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)
trigramPMItable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.pmi)), columns=['trigram','PMI']).sort_values(by='PMI', ascending=False)
bigramPMItable = bigramPMItable.head(20)
trigramPMItable = trigramPMItable.head(20) #вычленяем первые 20 лучших би- и триграмм
bigramPMItable.to_csv('Bigrams-PMI-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл
trigramPMItable.to_csv('Trigrams-PMI-test-20.csv',encoding='utf8', sep=';', index=False) #экспортируем в файл

In [ ]:
#RAKE

In [ ]:
!pip install pymorphy2
import pymorphy2 as pm
import codecs
import string

In [ ]:
m = pm.MorphAnalyzer()

In [ ]:
words = []
words2 = ""

In [ ]:
with open('Boyarsky.txt', 'r', encoding='utf-8') as f: #прописываем путь к корпусу (папка или файл)
    text = f.read()
    words = text.split()
    for i in range(len(words) - 1):
        if m.parse(words[i])[0].tag.POS == "ADVB" or m.parse(words[i])[0].tag.POS == "VERB" or m.parse(words[i][:-1])[0].tag.POS == "VERB" or m.parse(words[i])[0].tag.POS == "PRTS"or m.parse(words[i])[0].tag.POS == "INFN"or m.parse(words[i])[0].tag.POS == "COMP" or m.parse(words[i])[0].tag.POS == "ADJS" or m.parse(words[i])[0].tag.POS == "GRND"or m.parse(words[i])[0].tag.POS == "CONJ":
            words2 += " | " + words[i] + " | "
        elif (m.parse(words[i])[0].tag.POS == "NOUN" and m.parse(words[i])[0].tag.case == ("gent" or "accs")):
            words2 += words[i] + " | "
        elif m.parse(words[i])[0].tag.POS == "NOUN" and m.parse(words[i + 1])[0].tag.POS == ("NOUN" or "ADJF") and m.parse(words[i])[0].tag.case != m.parse(words[i + 1])[0].tag.case and m.parse(words[i + 1])[0].tag.case != "gent":
            words2 += words[i] + " | "
        elif (m.parse(words[i])[0].tag.POS == "NOUN" and m.parse(words[i])[0].tag.case != ("nomn" or "accs") and m.parse(words[i + 1])[0].tag.POS == "NOUN" and m.parse(words[i + 1])[0].tag.case == ("nomn" or "accs")):
            words2 += words[i] + " | "
        elif (m.parse(words[i])[0].tag.POS == None and m.parse(words[i][:-1])[0].tag.POS == None and m.parse(words[i][1:])[0].tag.POS == None):
            words2 += " | " + words[i] + " | "
        elif (m.parse(words[i])[0].tag.POS == "NOUN" and m.parse(words[i - 1])[0].tag.POS == "ADJF" or m.parse(words[i - 1])[0].tag.POS == "PRTF") and m.parse(words[i])[0].tag.case == m.parse(words[i + 1])[0].tag.case:
            words2 += words[i] + " | "
        else:
            words2 += words[i] + " "

In [ ]:
words2

In [ ]:
with open('Boyarsky.txt', 'w', encoding='utf-8') as f2:
    f2.write(words2)

In [ ]:
from __future__ import division
import operator
import nltk

In [ ]:
punct = open("swl.txt", 'r', encoding='utf-8').read()

In [ ]:
punct_list = nltk.word_tokenize(punct)

In [ ]:
stop = open("swl.txt", 'r', encoding='utf-8').read()

In [ ]:
pronouns = open("swl.txt", 'r', encoding='utf-8').read()

In [ ]:
preps = open("stopwords/preps.txt", 'r', encoding='utf-8').read()

In [ ]:
preps

In [ ]:
mystop = open("swl.txt", 'r', encoding='utf-8').read()

In [ ]:
#nltk.word_tokenize(preps.decode("utf8")) + nltk.word_tokenize(mystop.decode("utf8"))
nltk.word_tokenize(preps) + nltk.word_tokenize(mystop)

In [ ]:
stop_list = nltk.word_tokenize(stop) + nltk.word_tokenize(pronouns) + nltk.word_tokenize(preps) + nltk.word_tokenize(mystop)

In [ ]:
def isPunct(word):
  return len(word) == 1 and (word in string.punctuation or word in punct_list)

In [ ]:
def isNumeric(word):
  try:
    float(word) if '.' in word else int(word)
    return True
  except ValueError:
    return False

In [ ]:
def isInitial(word):
  return len(word) == 2 and word[1] == "."

In [ ]:
class RakeKeywordExtractor:

  def __init__(self):
    self.stopwords = set(nltk.corpus.stopwords.words())
    self.top_fraction = 1 # consider top third candidate keywords by score

  def _generate_candidate_keywords(self, sentences):
    phrase_list = []
    for sentence in sentences:
      words = map(lambda x: "|" if x in self.stopwords or x in stop_list or isNumeric(x) or isInitial(x) else x,
        nltk.word_tokenize(sentence.lower()))
      phrase = []
      for word in words:
        if word == "|" or isPunct(word):
          if len(phrase) > 0:
            phrase_list.append(phrase)
            phrase = []
        else:
          phrase.append(word)
    return phrase_list
## СТОП!!
  def _calculate_word_scores(self, phrase_list):
    word_freq = nltk.FreqDist()
    word_degree = nltk.FreqDist()
    for phrase in phrase_list:
      #degree = len(filter(lambda x: not isNumeric(x), phrase)) - 1
      degree = len(list(filter(lambda x: not isNumeric(x), phrase))) - 1
      for word in phrase:
        word_freq[word] += 1 ## word_fd.inc(word.lower()) with word_fd[word.lower()] += 1
        word_degree[word] += degree # other words
    for word in word_freq.keys():
      word_degree[word] = word_degree[word] + word_freq[word] # itself
    # word score = deg(w) / freq(w)
    word_scores = {}
    for word in word_freq.keys():
      word_scores[word] = word_degree[word] / word_freq[word]
    return word_scores

  def _calculate_phrase_scores(self, phrase_list, word_scores):
    phrase_scores = {}
    for phrase in phrase_list:
      phrase_score = 0
      for word in phrase:
        phrase_score += word_scores[word]
      phrase_scores[" ".join(phrase)] = phrase_score
    return phrase_scores

  def extract(self, text, incl_scores=False):
    sentences = nltk.sent_tokenize(text)
    phrase_list = self._generate_candidate_keywords(sentences)
    word_scores = self._calculate_word_scores(phrase_list)
    phrase_scores = self._calculate_phrase_scores(
      phrase_list, word_scores)
    #sorted_phrase_scores = sorted(phrase_scores.iteritems(),
    sorted_phrase_scores = sorted(phrase_scores.items(),
      key=operator.itemgetter(1), reverse=True)
    n_phrases = len(sorted_phrase_scores)
    if incl_scores:
      return sorted_phrase_scores[0:int(n_phrases/self.top_fraction)]
    else:
      return map(lambda x: x[0],
        sorted_phrase_scores[0:int(n_phrases/self.top_fraction)])

In [ ]:
rake = RakeKeywordExtractor()

In [ ]:
with open('D:\dialogues7.txt', 'r', encoding='utf-8') as f:
    txt = f.read()

In [ ]:
keywords = rake.extract(txt, incl_scores=True)

In [ ]:
for eachkwrd in keywords:
    print(eachkwrd[0], eachkwrd[1])

In [ ]:
keywords

In [ ]:
with open('rake_keywords.csv', 'w', encoding='utf-8') as f2:
    for eachkwrd in keywords:
        f2.write(str(eachkwrd[0])+";"+str(eachkwrd[1])+"\n")

In [ ]:
#KeyBERT
!pip install keybert
import keybert

In [ ]:
from keybert import KeyBERT

In [ ]:
with open('Boyarsky.txt', 'r', encoding='utf-8') as f:
    doc = f.read()

In [ ]:
kw_model = KeyBERT()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
keywords = kw_model.extract_keywords(doc)

In [ ]:
print(keywords)
#with open('KW keybert.txt', 'w', encoding='utf-8') as kb:
#  for i in keywords:
#    kb.write(str(keywords))

[('автоматический', 0.4882), ('информации', 0.4826), ('английской', 0.4822), ('источники', 0.4717), ('исследовательский', 0.4693), ('каневский2', 0.4671), ('боярский1', 0.4663), ('словосочетаний', 0.4572), ('исторические', 0.4563), ('математический', 0.4544), ('восстановлении', 0.4539), ('окончания', 0.4452), ('японский', 0.4452), ('изысканноромантические', 0.4425), ('российское', 0.4418), ('арестованный', 0.4402), ('механики', 0.4398), ('соскабливании', 0.4365), ('сочетаниями', 0.4355), ('описанные', 0.4338)]


In [ ]:
keywords = kw_model.extract_keywords(doc, top_n=20)
keywords

In [ ]:
with open('KeyBERT_top_20.csv', 'w', encoding='utf-8') as f2:
    for eachkwrd in keywords:
        f2.write(str(eachkwrd[0])+";"+str(eachkwrd[1])+"\n")

In [ ]:
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), top_n=20)
keywords

In [ ]:
with open('KeyBERT_top_20_bigram.csv', 'w', encoding='utf-8') as f2:
    for eachkwrd in keywords:
        f2.write(str(eachkwrd[0])+";"+str(eachkwrd[1])+"\n")

In [ ]:
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), top_n=20)
keywords

In [ ]:
with open('KeyBERT_top_20_trigram.csv', 'w', encoding='utf-8') as f2:
    for eachkwrd in keywords:
        f2.write(str(eachkwrd[0])+";"+str(eachkwrd[1])+"\n")

In [ ]:
#RuTermExtract

In [ ]:
!pip install rutermextract

In [ ]:
from rutermextract import TermExtractor

In [ ]:
term_extractor = TermExtractor()

In [ ]:
with open('Boyarsky.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
for term in term_extractor(text):
    print(term.normalized, term.count)

In [ ]:
with open('RuTermExtract_keywords.txt', 'w', encoding='utf-8') as f2:
    for term in term_extractor(text):
        f2.write(str(term.normalized)+";"+str(term.count)+"\n")

In [ ]:
#SpaCy

In [ ]:
!pip install spacy

In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz

In [ ]:
import spacy
from collections import Counter
from nltk.corpus import stopwords

In [ ]:
# Для SpaCy (дополнительно взяли стоп-слова из nltk, с ними результаты улучшились)
spacy_model = spacy.load('ru_core_news_sm')

In [ ]:
spacy_model.max_length = 2400000

In [ ]:
import nltk
nltk.download('stopwords')
nltk_stopwords = stopwords.words('russian')

In [ ]:
spacy_stopwords = spacy_model.Defaults.stop_words

In [ ]:
def extract_keywords_with_spacy(text):
    keywords = []
    doc = spacy_model(text.read())
    for token in doc:
        # Тексты уже очищены от знаков препинания и лемматизированы, тем не менее, остались некоторые символы,
        # которые нужно удалить
        if token.text not in spacy_stopwords and token.text not in nltk_stopwords and \
                token.text not in "`'«»...—-":
            if token.pos_ in ('ADJ', 'NOUN', 'VERB'):
                keywords.append(token.text)

    freq_word = Counter(keywords)
    max_freq = Counter(keywords).most_common(1)[0][1]
    for w in freq_word:
        freq_word[w] = round(freq_word[w] / max_freq, 3)

    freq_20 = freq_word.most_common(20)

    with open('spacykeywords.csv', 'w', encoding='utf-8') as f2:
        for term in freq_20:
            f2.write(str(term[0])+";"+str(term[1])+"\n")

    print(freq_20)

In [ ]:
with open('Boyarsky.txt', 'r', encoding='utf-8') as file:
    extract_keywords_with_spacy(file)

[('прил', 1.0), ('предлог', 0.81), ('сущ', 0.776), ('пад', 0.741), ('словосочетанием', 0.552), ('стоит', 0.552), ('род', 0.534), ('оборот', 0.517), ('следует', 0.5), ('словосочетания', 0.448), ('слов', 0.259), ('предложный', 0.259), ('словосочетаний', 0.172), ('слова', 0.172), ('глагол', 0.172), ('является', 0.155), ('словосочетание', 0.155), ('местоим', 0.155), ('союз', 0.155), ('оборотов', 0.138)]


In [ ]:
#Pullenti

In [ ]:
!pip install pullenti_wrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 9.1 MB/s eta 0:00:00


In [ ]:
from pullenti_wrapper.processor import Processor, GEO, ORGANIZATION, PERSON, PHONE, ADDRESS
from collections import Counter

In [ ]:
pullenti_processor = Processor([PERSON, ORGANIZATION, GEO, PHONE, ADDRESS])

In [ ]:
def extract_keywords_with_pullenti(text):
    keywords = []
    for text in text.read().splitlines():
        try:
            result = pullenti_processor(text)
            for match in result.walk():
                for key, value in match.referent.slots:
                    if key not in ('ISRELATIVE', 'SEX', 'NUMBER', 'HIGHER', 'PROFILE',
                                   'ALPHA2', 'POINTER', 'FROM', 'REF', 'ATTRIBUTE'):
                        if 'LASTNAME' in str(match.referent.slots) and 'FIRSTNAME' in str(match.referent.slots):
                            try:
                                keywords.append(f'{match.referent.firstname} {match.referent.lastname}'.lower())
                            except AttributeError:
                                continue
                        keywords.append(str(value).lower())
        except ValueError:
            continue

    freq_word = Counter(keywords)

    freq_20 = freq_word.most_common(20)

    with open('pullenti_keywords.csv', 'w', encoding='utf-8') as f2:
        for term in freq_20:
            f2.write(str(term[0])+";"+str(term[1])+"\n")

    print(freq_20)

In [ ]:
with open('Boyarsky.txt', 'r', encoding='utf-8') as file:
    extract_keywords_with_pullenti(file)


[('государство', 7), ('город', 6), ('рф', 3), ('российская федерация', 3), ('россия', 3), ('университет', 2), ("georeferent(label='geo', slots=[slot(key='name', value='санкт-петербург'), slot(key='type', value='город')])", 2), ('москва', 2), ('парламент', 2), ('к маркс', 2), ('михаил ходорковский', 2), ('санкт-петербург', 2), ('санкт петербургский национальный исследовательский университет информационных технологий', 1), ('университет итмо', 1), ('академия', 1), ('российская академия наук', 1), ('ран', 1), ("georeferent(label='geo', slots=[slot(key='alpha2', value='ru'), slot(key='name', value='рф'), slot(key='name', value='российская федерация'), slot(key='name', value='россия'), slot(key='type', value='государство')])", 1), ('институт', 1), ('санкт петербургский экономико математический институт', 1)]
